In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [2]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [3]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [4]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


PARA EMPEZAR ME TRAIGO LAS TABLAS DE TRANSFERENCIAS DE TFMKT Y WYSCOUT

In [5]:


# Consulta SQL
query2 = """SELECT * FROM ldp-data-wyscout.transfer_market.players_transfers_curated"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


  row_ind         player_name season transfer_date       team_from  \
0      10      Olivier Giroud  05/06    2005-07-01    Grenoble U19   
1      13  Zlatan Ibrahimović  98/99    1999-01-01    Malmö FF U19   
2      14          Raúl Baena  05/06    2005-07-01       Barça U16   
3      13              Castro  05/06    2005-07-01    FC Porto U17   
4      16        Stefan Savic  05/06    2005-09-02  Liefering Yth.   

          team_to market_value transfer_value contract_expiry  \
0      Grenoble B           NA             NA              NA   
1        Malmö FF           NA             NA              NA   
2       Barça U18           NA             NA              NA   
3    FC Porto U19           NA             NA              NA   
4  Salzburg Youth           NA             NA              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/olivier-giroud/p...  2024-09-08   
1  https://www.transfermarkt.com/zlatan-ibrahimov...

In [6]:
#df1=pd.read_csv('tfmkt.csv')

In [7]:
df1.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,10,Olivier Giroud,05/06,2005-07-01,Grenoble U19,Grenoble B,NA,NA,NA,https://www.transfermarkt.com/olivier-giroud/p...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,82442,https://www.transfermarkt.com/ac-mailand/start...,5,20050701582442
1,13,Zlatan Ibrahimović,98/99,1999-01-01,Malmö FF U19,Malmö FF,NA,NA,NA,https://www.transfermarkt.com/zlatan-ibrahimov...,2024-09-09,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,3455,https://www.transfermarkt.com/ac-mailand/start...,5,1999010153455
2,14,Raúl Baena,05/06,2005-07-01,Barça U16,Barça U18,NA,NA,NA,https://www.transfermarkt.com/raul-baena/profi...,2024-09-07,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,74557,https://www.transfermarkt.com/kitchee/startsei...,15979,200507011597974557
3,13,Castro,05/06,2005-07-01,FC Porto U17,FC Porto U19,NA,NA,NA,https://www.transfermarkt.com/castro/profil/sp...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,58267,https://www.transfermarkt.com/sc-braga/startse...,1075,20050701107558267
4,16,Stefan Savic,05/06,2005-09-02,Liefering Yth.,Salzburg Youth,NA,NA,NA,https://www.transfermarkt.com/stefan-savic/pro...,2023-10-01,2023-09-30 17:51:20+00:00,None,411713,https://www.transfermarkt.com/sv-rott/startsei...,24941,2005090224941411713
5,12,Akram Moghrabi,05/06,2005-07-01,Nejmeh SC U20,Nejmeh SC,NA,NA,NA,https://www.transfermarkt.com/akram-moghrabi/p...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,37043,https://www.transfermarkt.com/al-hikma/startse...,22199,200507012219937043
6,8,Pablo Lacoste,01/02,2002-01-01,Danubio U19,Defensor SC U19,NA,NA,NA,https://www.transfermarkt.com/pablo-lacoste/pr...,2024-09-07,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,142970,https://www.transfermarkt.com/ca-cerro/startse...,14806,2002010114806142970
7,16,Arnaud Bühler,01/02,2001-07-01,Yverdon Youth,Team Vaud U18,NA,NA,NA,https://www.transfermarkt.com/arnaud-buhler/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,10058,https://www.transfermarkt.com/fc-bavois/starts...,8650,20010701865010058
8,11,Django Warmerdam,03/04,2003-07-01,Foreholte Yth.,Ajax Youth,NA,0,NA,https://www.transfermarkt.com/django-warmerdam...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,288449,https://www.transfermarkt.com/fc-utrecht/start...,200,20030701200288449
9,21,Dmitriy Zinovich,05/06,2006-01-01,Shinnik U19,Shinnik II,NA,NA,NA,https://www.transfermarkt.com/dmitriy-zinovich...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,56572,https://www.transfermarkt.com/fk-murom/startse...,54924,200601015492456572


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101741 entries, 0 to 1101740
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   row_ind             1101741 non-null  object             
 1   player_name         1101741 non-null  object             
 2   season              1101741 non-null  object             
 3   transfer_date       1101741 non-null  object             
 4   team_from           1101741 non-null  object             
 5   team_to             1101741 non-null  object             
 6   market_value        1101741 non-null  object             
 7   transfer_value      1101741 non-null  object             
 8   contract_expiry     1101741 non-null  object             
 9   player_url          1101741 non-null  object             
 10  query_date          1101741 non-null  dbdate             
 11  timestamp           1101741 non-null  datetime64[us, UTC]
 12  

In [9]:
dftext = df1.to_csv('tfmkt.csv', index=False)

In [10]:
###FILTRO LA DE DICT TEAMS POR team_type="parent" and category="default" and gender = 'male' Y ADEMAS POR LOS PAISES QUE ESTAN EN LA DE CAPOLOGY PARA ACHICAR REGISTROS

query = """select * from ldp-data-wyscout.wyscout_curated.transfer_con_nombre;
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

                nombre  transferId  fromTeamId                fromTeamName  \
0  Herman Oen Paulsrud     2252070       24654        Grorud Idrettslag II   
1        Lucas Poulsen     2352906        7463                BK Frem 1886   
2         Osama Housni     2252131        7292                  Holmlia SK   
3         Ahmad Shaban     2376923       13294              Hapoel Acre FC   
4   Ryan Pereira Costa     2352994       23709  Os Belenenses SAD Under 19   

   toTeamId       toTeamName active   startDate     endDate        type value  \
0      7345        Grorud IL      0  2024-03-22        None    Transfer     0   
1     34555        Sundby BK      0  2024-08-01        None    Transfer     0   
2      7354    Nordstrand IF      0  2023-07-28  2024-03-05    Transfer     0   
3     13364  Hapoel Afula FC      0  2024-07-01        None    Transfer     0   
4      <NA>             None      0  2024-07-30        None  Free Agent     0   

  currency announceDate  playerId  
0      E

In [11]:
#df2=pd.read_csv('wyscout.csv')

In [12]:
# Reemplazar NaN con un valor por defecto, por ejemplo, -1 o 0
#df2['fromTeamId'] = df2['fromTeamId'].fillna(0).astype(int)
#df2['toTeamId'] = df2['toTeamId'].fillna(0).astype(int)


In [13]:
df2.head()

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,Herman Oen Paulsrud,2252070,24654,Grorud Idrettslag II,7345,Grorud IL,0,2024-03-22,None,Transfer,0,EUR,,1061921
1,Lucas Poulsen,2352906,7463,BK Frem 1886,34555,Sundby BK,0,2024-08-01,None,Transfer,0,EUR,,1061847
2,Osama Housni,2252131,7292,Holmlia SK,7354,Nordstrand IF,0,2023-07-28,2024-03-05,Transfer,0,EUR,,1061932
3,Ahmad Shaban,2376923,13294,Hapoel Acre FC,13364,Hapoel Afula FC,0,2024-07-01,None,Transfer,0,EUR,,1061976
4,Ryan Pereira Costa,2352994,23709,Os Belenenses SAD Under 19,<NA>,None,0,2024-07-30,None,Free Agent,0,EUR,,1061882


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        object
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  object
 4   toTeamId      Int64 
 5   toTeamName    object
 6   active        object
 7   startDate     object
 8   endDate       object
 9   type          object
 10  value         object
 11  currency      object
 12  announceDate  object
 13  playerId      Int64 
dtypes: Int64(4), object(10)
memory usage: 213.9+ MB


In [15]:
dftext2 = df2.to_csv('wyscout.csv',index=False)

In [16]:

# Consulta SQL
query2 = """SELECT * FROM ldp-data-wyscout.transfer_market.players_transfers_curated
                order by transfer_date desc limit 1000; """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df3 = client.query(query2).to_dataframe()
    print(df3.head())
except Exception as e:
    print(f"An error occurred: {e}")

  row_ind        player_name season transfer_date      team_from  \
0      10  Aleksandar Busnic  12/13            NA   FK Rad Youth   
1       7      Sizwe Mdlinzo  22/23            NA  Chippa United   
2       8       Chi-Kin Kwok  05/06            NA   Without Club   
3       2      Keleb Nwubani  20/21            NA  Brentford U18   
4       8           Leozinho  13/14            NA         Olaria   

            team_to market_value transfer_value contract_expiry  \
0           Rad U17           NA             NA              NA   
1            Marumo        3e+05             NA              NA   
2  Eastern Reserves           NA              0              NA   
3   TuS Erndtebrück           NA             NA              NA   
4             Macaé           NA             NA              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aleksandar-busni...  2024-02-07   
1  https://www.transfermarkt.com/sizwe-mdlinzo/pr...

In [17]:
df_sin_nulos = df3.dropna(subset=['transfer_date'])

In [18]:
df_sin_nulos.head()

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,10,Aleksandar Busnic,12/13,NA,FK Rad Youth,Rad U17,NA,NA,NA,https://www.transfermarkt.com/aleksandar-busni...,2024-02-07,2024-02-05 16:21:04+00:00,NA,399844,https://www.transfermarkt.com/fk-vojvodina-nov...,448,00000000448399844
1,7,Sizwe Mdlinzo,22/23,NA,Chippa United,Marumo,3e+05,NA,NA,https://www.transfermarkt.com/sizwe-mdlinzo/pr...,2024-09-09,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,346094,https://www.transfermarkt.com/tshakhuma-tsha-m...,61852,0000000061852346094
2,8,Chi-Kin Kwok,05/06,NA,Without Club,Eastern Reserves,NA,0,NA,https://www.transfermarkt.com/chi-kin-kwok/pro...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,362753,https://www.transfermarkt.com/shatin/startseit...,34344,0000000034344362753
3,2,Keleb Nwubani,20/21,NA,Brentford U18,TuS Erndtebrück,NA,NA,NA,https://www.transfermarkt.com/keleb-nwubani/pr...,2024-02-07,2024-02-05 16:21:53+00:00,NA,933765,https://www.transfermarkt.com/preussen-munster...,7019,000000007019933765
4,8,Leozinho,13/14,NA,Olaria,Macaé,NA,NA,NA,https://www.transfermarkt.com/leozinho/profil/...,2024-09-08,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,354028,https://www.transfermarkt.com/wyvern-kariya/st...,88269,0000000088269354028


In [19]:
# Convertir la columna transfer_date a tipo datetime si no lo está
df1['transfer_date'] = pd.to_datetime(df1['transfer_date'], errors='coerce')

# Obtener el rango de valores de la columna transfer_date
min_date = df1['transfer_date'].min()
max_date = df1['transfer_date'].max()

print(f"El rango de valores de la columna transfer_date es desde {min_date} hasta {max_date}.")

El rango de valores de la columna transfer_date es desde 1939-07-01 00:00:00 hasta 2033-03-01 00:00:00.


In [20]:
# Convertir las columnas startDate y endDate a tipo datetime si no lo están
df2['startDate'] = pd.to_datetime(df2['startDate'], errors='coerce')
df2['endDate'] = pd.to_datetime(df2['endDate'], errors='coerce')

# Obtener el rango de valores de las columnas startDate y endDate
min_start_date = df2['startDate'].min()
max_start_date = df2['startDate'].max()
min_end_date = df2['endDate'].min()
max_end_date = df2['endDate'].max()

print(f"El rango de valores de la columna startDate es desde {min_start_date} hasta {max_start_date}.")
print(f"El rango de valores de la columna endDate es desde {min_end_date} hasta {max_end_date}.")

El rango de valores de la columna startDate es desde 1982-06-24 00:00:00 hasta 2028-09-01 00:00:00.
El rango de valores de la columna endDate es desde 1985-07-01 00:00:00 hasta 2094-07-01 00:00:00.


In [21]:
df3.head(40)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,10,Aleksandar Busnic,12/13,NA,FK Rad Youth,Rad U17,NA,NA,NA,https://www.transfermarkt.com/aleksandar-busni...,2024-02-07,2024-02-05 16:21:04+00:00,NA,399844,https://www.transfermarkt.com/fk-vojvodina-nov...,448,00000000448399844
1,7,Sizwe Mdlinzo,22/23,NA,Chippa United,Marumo,3e+05,NA,NA,https://www.transfermarkt.com/sizwe-mdlinzo/pr...,2024-09-09,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,346094,https://www.transfermarkt.com/tshakhuma-tsha-m...,61852,0000000061852346094
2,8,Chi-Kin Kwok,05/06,NA,Without Club,Eastern Reserves,NA,0,NA,https://www.transfermarkt.com/chi-kin-kwok/pro...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,362753,https://www.transfermarkt.com/shatin/startseit...,34344,0000000034344362753
3,2,Keleb Nwubani,20/21,NA,Brentford U18,TuS Erndtebrück,NA,NA,NA,https://www.transfermarkt.com/keleb-nwubani/pr...,2024-02-07,2024-02-05 16:21:53+00:00,NA,933765,https://www.transfermarkt.com/preussen-munster...,7019,000000007019933765
4,8,Leozinho,13/14,NA,Olaria,Macaé,NA,NA,NA,https://www.transfermarkt.com/leozinho/profil/...,2024-09-08,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,354028,https://www.transfermarkt.com/wyvern-kariya/st...,88269,0000000088269354028
5,4,Christian Andrade,19/20,NA,Millonarios U20,Medellín U20,NA,NA,NA,https://www.transfermarkt.com/christian-andrad...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,491534,https://www.transfermarkt.com/envigado-fc/star...,7136,000000007136491534
6,21,Claudio Zappa,13/14,NA,AlbinoLeffe U19,Beira-Mar U19,NA,0,NA,https://www.transfermarkt.com/claudio-zappa/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,291433,https://www.transfermarkt.com/ssd-real-calepin...,82583,0000000082583291433
7,1,Pierre Bourdet,20/21,NA,ESC Longueau,Retired,NA,NA,NA,https://www.transfermarkt.com/pierre-bourdet/p...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,650440,https://www.transfermarkt.com/esc-longueau/sta...,63352,0000000063352650440
8,4,Austine Ogunye,22/23,NA,Sunshine Stars,Bendel Ins.,NA,NA,NA,https://www.transfermarkt.com/austine-ogunye/p...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,536610,https://www.transfermarkt.com/bendel-insurance...,2979,000000002979536610
9,2,Onyeka Nwaiwu,22/23,NA,Sunshine Stars,Akwa United,NA,NA,NA,https://www.transfermarkt.com/onyeka-nwaiwu/pr...,2024-09-09,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,986587,https://www.transfermarkt.com/akwa-united/star...,16953,0000000016953986587


In [22]:

query = """select * from ldp-data-wyscout.wyscout_curated.transfer_con_nombre
                    order by startDate desc
                    limit 1000;"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df4 = client.query(query).to_dataframe()
    print(df4.head())
except Exception as e:
    print(f"An error occurred: {e}")

                 nombre  transferId  fromTeamId                  fromTeamName  \
0         János Bodrogi     1203917       23191  Nagybaracska-Csátalja Híd SC   
1      Abdullah Soğucak     2439880       26846         Altınordu SK Under 17   
2     Viet Hoang Vo Huu     2385629       16624           Hồ Chí Minh City FC   
3  Thanh Long Phan Nhật     2385601       16624           Hồ Chí Minh City FC   
4     Vinh Nguyen Hoang     2385674       16624           Hồ Chí Minh City FC   

   toTeamId                    toTeamName active   startDate endDate  \
0      <NA>                          None      0  2028-09-01     NaT   
1     71478  Efeler 09 Spor Futbol Kulubu      0  2027-08-01     NaT   
2      <NA>                                    0  2027-06-30     NaT   
3      <NA>                                    0  2027-06-30     NaT   
4      <NA>                                    0  2027-06-30     NaT   

             type value currency announceDate  playerId  
0         Retired     

In [23]:
#df_sin_nulos = df4.dropna(subset=['nombre'])

In [24]:
df4.head(40)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,János Bodrogi,1203917,23191,Nagybaracska-Csátalja Híd SC,<NA>,None,0,2028-09-01,NaT,Retired,0,EUR,,352224
1,Abdullah Soğucak,2439880,26846,Altınordu SK Under 17,71478,Efeler 09 Spor Futbol Kulubu,0,2027-08-01,NaT,Transfer,0,EUR,,601447
2,Viet Hoang Vo Huu,2385629,16624,Hồ Chí Minh City FC,<NA>,,0,2027-06-30,NaT,Back from Loan,0,EUR,,895753
3,Thanh Long Phan Nhật,2385601,16624,Hồ Chí Minh City FC,<NA>,,0,2027-06-30,NaT,Back from Loan,0,EUR,,1061713
4,Vinh Nguyen Hoang,2385674,16624,Hồ Chí Minh City FC,<NA>,,0,2027-06-30,NaT,Back from Loan,0,EUR,,895752
5,Peter Molinari,2237470,69751,New York City FC II,32556,New York City FC,0,2027-01-01,NaT,Back from Loan,0,EUR,,879745
6,Gabriel Loureiro,2177732,60297,FC Cesarense Under 19,75235,FC Lusitania de Lourosa Under,0,2026-07-01,NaT,Transfer,0,EUR,,1039597
7,Serhiy Zaporozhian,2358673,<NA>,,76383,Livyi Bereh U19,0,2026-07-01,NaT,Free Transfer,0,EUR,,1096099
8,Justin Armando Lerma Soliz,2272135,15788,CSD Independiente del Valle,2447,BV Borussia 09 Dortmund,0,2026-07-01,NaT,Transfer,0,EUR,,960284
9,Ahmed Khaled Gomaa,2276641,69129,Modern Future FC,16041,Al Ahly SC,0,2026-06-30,NaT,Back from Loan,0,EUR,,949832


In [25]:
# Convertir la columna transfer_date a tipo datetime si no lo está
df1['transfer_date'] = pd.to_datetime(df1['transfer_date'], errors='coerce')

# Obtener el rango de valores de la columna transfer_date
min_date = df1['transfer_date'].min()
max_date = df1['transfer_date'].max()

print(f"El rango de valores de la columna transfer_date es desde {min_date} hasta {max_date}.")

El rango de valores de la columna transfer_date es desde 1939-07-01 00:00:00 hasta 2033-03-01 00:00:00.


In [26]:
# Convertir las columnas startDate y endDate a tipo datetime si no lo están
df2['startDate'] = pd.to_datetime(df2['startDate'], errors='coerce')
df2['endDate'] = pd.to_datetime(df2['endDate'], errors='coerce')

# Obtener el rango de valores de las columnas startDate y endDate
min_start_date = df2['startDate'].min()
max_start_date = df2['startDate'].max()
min_end_date = df2['endDate'].min()
max_end_date = df2['endDate'].max()

print(f"El rango de valores de la columna startDate es desde {min_start_date} hasta {max_start_date}.")
print(f"El rango de valores de la columna endDate es desde {min_end_date} hasta {max_end_date}.")

El rango de valores de la columna startDate es desde 1982-06-24 00:00:00 hasta 2028-09-01 00:00:00.
El rango de valores de la columna endDate es desde 1985-07-01 00:00:00 hasta 2094-07-01 00:00:00.


In [43]:
# Probar con una muestra de 1000 filas de cada DataFrame
df1_sample = df1.sample(n=1000, random_state=1)
df2_sample = df2.sample(n=1000, random_state=1)

# Normalizar las columnas en ambos DataFrames para facilitar la comparación
df1_sample['player_name'] = df1_sample['player_name'].str.lower().str.strip()
df1_sample['team_from'] = df1_sample['team_from'].str.lower().str.strip()
df1_sample['team_to'] = df1_sample['team_to'].str.lower().str.strip()

df2_sample['nombre'] = df2_sample['nombre'].str.lower().str.strip()
df2_sample['fromTeamName'] = df2_sample['fromTeamName'].str.lower().str.strip()
df2_sample['toTeamName'] = df2_sample['toTeamName'].str.lower().str.strip()

# Encontrar el mejor match para los jugadores y equipos entre df1_sample y df2_sample
df1_sample['player_match'] = df1_sample['player_name'].apply(lambda x: get_best_match(x, df2_sample['nombre'].tolist()))
df1_sample['fromTeam_match'] = df1_sample['team_from'].apply(lambda x: get_best_match(x, df2_sample['fromTeamName'].tolist()))
df1_sample['toTeam_match'] = df1_sample['team_to'].apply(lambda x: get_best_match(x, df2_sample['toTeamName'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2_sample
player_to_id_sample = dict(zip(df2_sample['nombre'], df2_sample['playerId']))
fromTeam_to_id_sample = dict(zip(df2_sample['fromTeamName'], df2_sample['fromTeamId']))
toTeam_to_id_sample = dict(zip(df2_sample['toTeamName'], df2_sample['toTeamId']))

df1_sample['playerId'] = df1_sample['player_match'].map(player_to_id_sample)

# Definir final_columns
final_columns = [
    'player_name', 'team_from', 'team_to', 'season', 'transfer_date', 'market_value', 'transfer_value', 'contract_expiry', 'player_url', 'player_id', 'player_transfer_id'
]
df1_sample['fromTeamId'] = df1_sample['fromTeam_match'].map(fromTeam_to_id_sample)
df1_sample['toTeamId'] = df1_sample['toTeam_match'].map(toTeam_to_id_sample)

# Merge basado en las columnas 'playerId', 'fromTeamId', y 'toTeamId'
merged_df_sample = pd.merge(df1_sample, df2_sample, left_on=['playerId', 'fromTeamId', 'toTeamId'], right_on=['playerId', 'fromTeamId', 'toTeamId'], how='left', suffixes=('_df1', '_df2'))

# Seleccionar solo las columnas presentes en merged_df_sample
final_columns_existing_sample = [col for col in final_columns if col in merged_df_sample.columns]
df_final_sample = merged_df_sample[final_columns_existing_sample]

# Guardar o imprimir el DataFrame final
df_final_sample.to_csv('merged_transfers_sample.csv', index=False)
print(df_final_sample.head())



         player_name       team_from         team_to season transfer_date  \
0     ben hinchliffe   bamber bridge   northwich vic  11/12    2011-07-01   
1       valon ahmedi    without club  inter zapresic  19/20    2019-10-01   
2    francis bolland  leobendorf jgd   ruppthal yth.  08/09    2009-01-01   
3  aleksandr kulikov    without club     neftekhimik  17/18    2017-07-11   
4      sigurd grønli    tromsø il ii          tromsø  17/18    2017-10-31   

  market_value transfer_value contract_expiry  \
0           NA              0              NA   
1        6e+05             NA              NA   
2           NA             NA              NA   
3        1e+05             NA              NA   
4           NA             NA              NA   

                                          player_url player_id  \
0  https://www.transfermarkt.com/ben-hinchliffe/p...     49597   
1  https://www.transfermarkt.com/valon-ahmedi/pro...    226178   
2  https://www.transfermarkt.com/francis-bol

In [42]:
df_final_sample.head(50)

,player_name,team_from,team_to,season,transfer_date,market_value,transfer_value,contract_expiry,player_url,player_id,player_transfer_id,transferId
0,ben hinchliffe,bamber bridge,northwich vic,11/12,2011-07-01,NA,0,NA,https://www.transfermarkt.com/ben-hinchliffe/p...,49597,20110701109849597,<NA>
1,valon ahmedi,without club,inter zapresic,19/20,2019-10-01,6e+05,NA,NA,https://www.transfermarkt.com/valon-ahmedi/pro...,226178,2019100110695226178,<NA>
2,francis bolland,leobendorf jgd,ruppthal yth.,08/09,2009-01-01,NA,NA,NA,https://www.transfermarkt.com/francis-bolland/...,520043,2009010119578520043,<NA>
3,aleksandr kulikov,without club,neftekhimik,17/18,2017-07-11,1e+05,NA,NA,https://www.transfermarkt.com/aleksandr-kuliko...,73848,20170711324373848,<NA>
4,sigurd grønli,tromsø il ii,tromsø,17/18,2017-10-31,NA,NA,NA,https://www.transfermarkt.com/sigurd-gronli/pr...,548918,201710311057548918,<NA>
5,uros racanovic,bijeljina u19,jedinstvo b.,18/19,2019-01-01,NA,0,NA,https://www.transfermarkt.com/uros-racanovic/p...,586119,2019010130158586119,<NA>
6,amir natcho,barça u19,cska moscow ii,15/16,2015-08-31,1e+05,0,2017-06-30,https://www.transfermarkt.com/amir-natcho/prof...,330059,201508313690330059,<NA>
7,cavafe,cd tudelano,fc dornbirn,22/23,2022-07-01,50000,0,NA,https://www.transfermarkt.com/cavafe/profil/sp...,675890,202207017448675890,<NA>
8,robinho,novo hamburgo,ceará sc,15/16,2016-04-18,1e+05,NA,2016-05-15,https://www.transfermarkt.com/robinho/profil/s...,347184,201604189030347184,<NA>
9,frank mulhern,farsley,scarborough,23/24,2023-07-01,NA,0,NA,https://www.transfermarkt.com/frank-mulhern/pr...,343179,202307013818343179,<NA>


In [47]:
df1_sample.head(50)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,...,player_id,teams_url,team_id,player_transfer_id,player_match,fromTeam_match,toTeam_match,playerId,fromTeamId,toTeamId
79501,3,ben hinchliffe,11/12,2011-07-01,bamber bridge,northwich vic,NA,0,NA,https://www.transfermarkt.com/ben-hinchliffe/p...,...,49597,https://www.transfermarkt.com/stockport-county...,1098,20110701109849597,None,None,None,NaN,<NA>,<NA>
498435,4,valon ahmedi,19/20,2019-10-01,without club,inter zapresic,6e+05,NA,NA,https://www.transfermarkt.com/valon-ahmedi/pro...,...,226178,https://www.transfermarkt.com/shakhtjor-soligo...,10695,2019100110695226178,None,None,None,NaN,<NA>,<NA>
12057,9,francis bolland,08/09,2009-01-01,leobendorf jgd,ruppthal yth.,NA,NA,NA,https://www.transfermarkt.com/francis-bolland/...,...,520043,https://www.transfermarkt.com/fc-marchfeld-don...,19578,2009010119578520043,None,None,None,NaN,<NA>,<NA>
311677,4,aleksandr kulikov,17/18,2017-07-11,without club,neftekhimik,1e+05,NA,NA,https://www.transfermarkt.com/aleksandr-kuliko...,...,73848,https://www.transfermarkt.com/kamaz-naberezhny...,3243,20170711324373848,aleksandr yushin,None,None,391989.0,<NA>,<NA>
269450,7,sigurd grønli,17/18,2017-10-31,tromsø il ii,tromsø,NA,NA,NA,https://www.transfermarkt.com/sigurd-gronli/pr...,...,548918,https://www.transfermarkt.com/bryne-fk/startse...,1057,201710311057548918,None,None,atromitos,NaN,<NA>,12536
399034,5,uros racanovic,18/19,2019-01-01,bijeljina u19,jedinstvo b.,NA,0,NA,https://www.transfermarkt.com/uros-racanovic/p...,...,586119,https://www.transfermarkt.com/fk-po-shy-dri-sh...,30158,2019010130158586119,None,None,None,NaN,<NA>,<NA>
215353,12,amir natcho,15/16,2015-08-31,barça u19,cska moscow ii,1e+05,0,2017-06-30,https://www.transfermarkt.com/amir-natcho/prof...,...,330059,https://www.transfermarkt.com/ska-khabarovsk/s...,3690,201508313690330059,None,None,None,NaN,<NA>,<NA>
856326,2,cavafe,22/23,2022-07-01,cd tudelano,fc dornbirn,50000,0,NA,https://www.transfermarkt.com/cavafe/profil/sp...,...,675890,https://www.transfermarkt.com/fc-dornbirn/star...,7448,202207017448675890,None,None,None,NaN,<NA>,<NA>
246308,11,robinho,15/16,2016-04-18,novo hamburgo,ceará sc,1e+05,NA,2016-05-15,https://www.transfermarkt.com/robinho/profil/s...,...,347184,https://www.transfermarkt.com/botafogo-futebol...,9030,201604189030347184,None,None,None,NaN,<NA>,<NA>
998566,1,frank mulhern,23/24,2023-07-01,farsley,scarborough,NA,0,NA,https://www.transfermarkt.com/frank-mulhern/pr...,...,343179,https://www.transfermarkt.com/farsley-celtic/s...,3818,202307013818343179,None,None,None,NaN,<NA>,<NA>


In [48]:
df2_sample.head(50)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
535827,nicolás mauricio zalazar,650661,27832,san lorenzo res.,12055,atlético rafaela,0,2018-07-30,2019-06-30,Loan,0,EUR,,481907
158659,jefersson sabino dos santos,716480,6081,comercial,<NA>,,0,2018-12-01,NaT,Free Agent,0,EUR,,43600
338246,luca romanini,1330574,<NA>,None,3669,as pro piacenza,0,2014-07-01,2016-08-30,Free Transfer,0,EUR,,415362
562274,pierre alphonse diedhiou,1680284,5084,kvc westerlo,5241,ksk heist,0,2022-07-01,NaT,Transfer,0,EUR,,637220
1190170,caden dosmann,2335021,<NA>,,27997,grand canyon lopes,0,2024-07-01,NaT,Free Transfer,0,EUR,,1088671
1625326,ten miyagi,1862095,12875,v-varen nagasaki,12812,kawasaki frontale,0,2024-01-31,NaT,Back from Loan,0,EUR,,677015
850369,léonie multari,500229,28134,olympique de marseille,60083,football club fleury 91,1,2017-07-01,2019-07-01,Free Transfer,0,,2017-08-08,292278
588259,tommaso fermi,1019104,<NA>,,<NA>,reggiana u17,0,2020-07-01,NaT,Unknown,0,EUR,,692765
979830,aryan tajbakhsh,260975,1833,wealdstone fc,1726,crawley town fc,1,2017-03-19,NaT,Back from Loan,0,,2017-02-19,286409
194095,basil abdulrahman n alhedaif,2121025,<NA>,None,17012,al-ittihad club,0,2023-06-30,2023-09-01,Unknown,0,EUR,,645253


In [44]:
df_final_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   player_name         1000 non-null   object        
 1   team_from           1000 non-null   object        
 2   team_to             1000 non-null   object        
 3   season              1000 non-null   object        
 4   transfer_date       1000 non-null   datetime64[ns]
 5   market_value        1000 non-null   object        
 6   transfer_value      1000 non-null   object        
 7   contract_expiry     1000 non-null   object        
 8   player_url          1000 non-null   object        
 9   player_id           1000 non-null   object        
 10  player_transfer_id  1000 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 86.1+ KB


In [33]:
# Normalizar las columnas en ambos DataFrames para facilitar la comparación
df1['player_name'] = df1['player_name'].str.lower().str.strip()
df1['team_from'] = df1['team_from'].str.lower().str.strip()
df1['team_to'] = df1['team_to'].str.lower().str.strip()

df2['nombre'] = df2['nombre'].str.lower().str.strip()
df2['fromTeamName'] = df2['fromTeamName'].str.lower().str.strip()
df2['toTeamName'] = df2['toTeamName'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    if pd.isna(value):  # Manejar valores NaN
        return None
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 70 else None

# Encontrar el mejor match para los jugadores y equipos entre df1 y df2
df1['player_match'] = df1['player_name'].apply(lambda x: get_best_match(x, df2['nombre'].tolist()))
df1['fromTeam_match'] = df1['team_from'].apply(lambda x: get_best_match(x, df2['fromTeamName'].tolist()))
df1['toTeam_match'] = df1['team_to'].apply(lambda x: get_best_match(x, df2['toTeamName'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2
player_to_id = dict(zip(df2['nombre'], df2['playerId']))
fromTeam_to_id = dict(zip(df2['fromTeamName'], df2['fromTeamId']))
toTeam_to_id = dict(zip(df2['toTeamName'], df2['toTeamId']))

df1['playerId'] = df1['player_match'].map(player_to_id)
df1['fromTeamId'] = df1['fromTeam_match'].map(fromTeam_to_id)
df1['toTeamId'] = df1['toTeam_match'].map(toTeam_to_id)

# Merge basado en las columnas 'playerId', 'fromTeamId', y 'toTeamId'
merged_df = pd.merge(df1, df2, left_on=['playerId', 'fromTeamId', 'toTeamId'], right_on=['playerId', 'fromTeamId', 'toTeamId'], how='left', suffixes=('_df1', '_df2'))

# Seleccionar las columnas deseadas
final_columns = [
    'player_name', 'team_from', 'team_to', 'season', 'transfer_date', 'market_value', 'transfer_value', 'contract_expiry', 'player_url', 'player_id', 'player_transfer_id',
    'transferId', 'fromTeamId', 'toTeamId', 'startDate', 'endDate', 'type', 'value', 'currency', 'playerId'
]

# Asegúrate de que las columnas existan en el DataFrame
missing_columns = [col for col in final_columns if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns_existing = [col for col in final_columns if col in merged_df.columns]
df_final = merged_df[final_columns_existing]

# Guardar o imprimir el DataFrame final
df_final.to_csv('merged_transfers.csv', index=False)
print(df_final.head())


DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100